**Load the required libraries and data**

In [ ]:
# # Load the required libraries 
# import pandas as pd
# import numpy as np
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, precision_recall_curve
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
# # --- Assumptions ---
# # pred_df should have columns: filename, predicted_label, confidence
# # anno_df should have columns: filename, label
# # You may need to adjust column names as per your actual data

# # Merge on filename (or image_id if more appropriate)
# merged = pd.merge(pred_df, anno_df[['filename', 'label']], on='filename', how='inner', suffixes=('_pred', '_true'))

# # If pred_df has a confidence column, use it for thresholding
# thresholds = np.linspace(0, 1, 21)
# metrics = []

# for thresh in thresholds:
#     # Assume binary classification: 1 = defect, 0 = no defect
#     # Adjust as needed for multiclass
#     merged['pred_bin'] = (merged['confidence'] >= thresh).astype(int)
#     merged['true_bin'] = (merged['label'] != 'no defect').astype(int)  # Adjust if your negative class is named differently

#     acc = accuracy_score(merged['true_bin'], merged['pred_bin'])
#     prec = precision_score(merged['true_bin'], merged['pred_bin'], zero_division=0)
#     rec = recall_score(merged['true_bin'], merged['pred_bin'], zero_division=0)
#     f1 = f1_score(merged['true_bin'], merged['pred_bin'], zero_division=0)
#     metrics.append([thresh, acc, prec, rec, f1])

# metrics_df = pd.DataFrame(metrics, columns=['Threshold', 'Accuracy', 'Precision', 'Recall', 'F1'])

# # Plot metrics vs. threshold
# plt.figure(figsize=(10,6))
# plt.plot(metrics_df['Threshold'], metrics_df['Accuracy'], label='Accuracy')
# plt.plot(metrics_df['Threshold'], metrics_df['Precision'], label='Precision')
# plt.plot(metrics_df['Threshold'], metrics_df['Recall'], label='Recall')
# plt.plot(metrics_df['Threshold'], metrics_df['F1'], label='F1-score')
# plt.xlabel('Confidence Threshold')
# plt.ylabel('Score')
# plt.title('Metrics vs. Confidence Threshold')
# plt.legend()
# plt.show()

# # Confusion matrix at optimal threshold (e.g., max F1)
# best_idx = metrics_df['F1'].idxmax()
# best_thresh = metrics_df.loc[best_idx, 'Threshold']
# merged['pred_bin'] = (merged['confidence'] >= best_thresh).astype(int)
# cm = confusion_matrix(merged['true_bin'], merged['pred_bin'])

# plt.figure(figsize=(5,4))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title(f'Confusion Matrix (Threshold={best_thresh:.2f})')
# plt.show()

# # ROC Curve
# fpr, tpr, _ = roc_curve(merged['true_bin'], merged['confidence'])
# roc_auc = auc(fpr, tpr)
# plt.figure(figsize=(6,5))
# plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
# plt.plot([0,1], [0,1], 'k--')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.legend()
# plt.show()

# # Precision-Recall Curve
# prec, rec, _ = precision_recall_curve(merged['true_bin'], merged['confidence'])
# plt.figure(figsize=(6,5))
# plt.plot(rec, prec, label='Precision-Recall curve')
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.title('Precision-Recall Curve')
# plt.legend()
# plt.show()
# # ...existing code...

In [ ]:
import pandas as pd
import ast
from shapely.geometry import Polygon

In [ ]:
# Load data
anno_df = pd.read_csv('../data/clean-data/anno_df_clean.csv')
pred_df = pd.read_csv('../data/clean-data/pred_df_clean.csv')

In [ ]:
anno_df["xy"] = anno_df["xy"].apply(ast.literal_eval)
pred_df["xy"] = pred_df["xy"].apply(ast.literal_eval)

In [ ]:
def parse_xy(xy_str):
    # Implement parsing logic depending on your format
    # Example: if xy_str is 'x1,y1;x2,y2;...'
    return [tuple(map(float, point.split(','))) for point in xy_str.split(';')]

# def iou(poly1, poly2):
#     # Compute Intersection over Union using shapely
#     p1 = Polygon(poly1)
#     p2 = Polygon(poly2)
#     if not p1.is_valid or not p2.is_valid:
#         return 0.0
#     inter = p1.intersection(p2).area
#     union = p1.union(p2).area
#     return inter / union if union > 0 else 0

def iou(poly1, poly2):
    # Convert flat list to list of (x, y) tuples if needed
    if isinstance(poly1[0], (int, float)):
        poly1 = list(zip(poly1[::2], poly1[1::2]))
    if isinstance(poly2[0], (int, float)):
        poly2 = list(zip(poly2[::2], poly2[1::2]))
    p1 = Polygon(poly1)
    p2 = Polygon(poly2)
    if not p1.is_valid or not p2.is_valid:
        return 0
    inter = p1.intersection(p2).area
    union = p1.union(p2).area
    return inter / union if union > 0 else 0

In [ ]:
# Set IoU threshold for a match
IOU_THRESHOLD = 0.5

TP = 0
FP = 0
FN = 0

In [ ]:
for image_id in anno_df['image_id'].unique():
    gt_polys = anno_df[anno_df['image_id'] == image_id]['xy'].apply(parse_xy).tolist()
    pred_polys = pred_df[pred_df['image_id'] == image_id]['xy'].apply(parse_xy).tolist()
    
    matched_gt = set()
    matched_pred = set()
    
    for pred_idx, pred_poly in enumerate(pred_polys):
        found_match = False
        for gt_idx, gt_poly in enumerate(gt_polys):
            if gt_idx in matched_gt:
                continue
            if iou(pred_poly, gt_poly) >= IOU_THRESHOLD:
                TP += 1
                matched_gt.add(gt_idx)
                matched_pred.add(pred_idx)
                found_match = True
                break
        if not found_match:
            FP += 1
    FN += len(gt_polys) - len(matched_gt)

In [ ]:
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

In [ ]:
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

# New Implementation

In [14]:
import ast
import pandas as pd
from shapely.geometry import Polygon

In [5]:
# Load the cleaned CSVs
anno_df = pd.read_csv('../data/clean-data/anno_df_clean.csv')   # Manual annotations
pred_df = pd.read_csv('../data/clean-data/pred_df_clean.csv')   # Model predictions

In [6]:
# Keep only predictions for images that have manual annotations
filtered_pred_df = pred_df[pred_df['image_id'].isin(anno_df['image_id'])]

filtered_pred_df.head()

,image_id,prediction_id,confidence,polygon_id,xy
0,3165,23699,0.431523,1937097,"37.578460693359375,13.887542724609375,37.31016..."
1,3165,23699,0.603884,1937098,"231.45407104492188,9.646164820744442,225.45407..."
2,3165,23699,0.244653,1937099,"318.3146711077009,33.736045837402344,316.76688..."
3,3165,23699,0.916268,1937100,"395.9158020019531,82.97906857445126,387.915802..."
4,3165,23699,0.366087,1937101,"337.0517883300781,84.71218928584346,332.051788..."


In [9]:
anno_df.head()

,filename,image_id,id,xy,x,y
0,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,...","[1354.0, 1330.52, 1330.52, 1350.087, 1357.913,...","[1377.48, 1389.22, 1412.699, 1424.439, 1420.52..."
1,1g_04.png,624,175664,"223.058,1804.029,199.578,1800.116,191.751,1815...","[223.058, 199.578, 191.751, 203.491, 223.058, ...","[1804.029, 1800.116, 1815.769, 1831.422, 1835...."
2,1g_04.png,624,175634,"1549.665,1005.717,1545.751,1013.543,1541.838,1...","[1549.665, 1545.751, 1541.838, 1541.838, 1557....","[1005.717, 1013.543, 1021.37, 1037.023, 1037.0..."
3,1g_04.png,624,175654,"661.347,1678.803,637.867,1667.064,614.387,1674...","[661.347, 637.867, 614.387, 622.214, 630.04, 6...","[1678.803, 1667.064, 1674.89, 1698.37, 1694.45..."
4,1g_04.png,624,175633,"1784.462,1897.948,1796.202,1886.208,1804.029,1...","[1784.462, 1796.202, 1804.029, 1780.549, 1772....","[1897.948, 1886.208, 1870.555, 1858.815, 1862...."


In [10]:
# Get the features that will matter from the manual 
anno_df = anno_df[['filename', 'image_id', 'id', 'xy']]
anno_df.head()

,filename,image_id,id,xy
0,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,..."
1,1g_04.png,624,175664,"223.058,1804.029,199.578,1800.116,191.751,1815..."
2,1g_04.png,624,175634,"1549.665,1005.717,1545.751,1013.543,1541.838,1..."
3,1g_04.png,624,175654,"661.347,1678.803,637.867,1667.064,614.387,1674..."
4,1g_04.png,624,175633,"1784.462,1897.948,1796.202,1886.208,1804.029,1..."


In [11]:
# Merge on 'image_id' to pair manual and predicted for each image
merged_df = pd.merge(
    anno_df, filtered_pred_df,
    on='image_id',
    suffixes=('_manual', '_predicted')
)

# Preview the edits 
merged_df.head()

,filename,image_id,id,xy_manual,prediction_id,confidence,polygon_id,xy_predicted
0,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,...",23746,0.459061,1943470,"529.6653366088867,394.1354550962095,522.665336..."
1,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,...",23746,0.862112,1943471,"564.6947244497446,1107.9412689208984,556.54087..."
2,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,...",23746,0.455114,1943472,"548.7361194776452,1158.1390075683594,550.40501..."
3,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,...",23746,0.256400,1943473,"524.8235282897949,1337.475,535.8235282897949,1..."
4,1g_04.png,624,175683,"1354,1377.48,1330.52,1389.22,1330.52,1412.699,...",23746,0.725020,1943474,"506.43187522888184,1372.9502636137463,500.7483..."


In [13]:
# Pair the xy coordinates for shapely 

def parse_xy(xy_str):
    # If already a list, return as is
    if isinstance(xy_str, list):
        return xy_str
    # If string, parse to list of tuples
    coords = ast.literal_eval(xy_str) if isinstance(xy_str, str) else xy_str
    # If flat list, pair into tuples
    if coords and isinstance(coords[0], (int, float)):
        return [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]
    return coords

merged_df['xy_manual'] = merged_df['xy_manual'].apply(parse_xy)
merged_df['xy_predicted'] = merged_df['xy_predicted'].apply(parse_xy)

# Preview the edits 
merged_df.head()

,filename,image_id,id,xy_manual,prediction_id,confidence,polygon_id,xy_predicted
0,1g_04.png,624,175683,"[(1354, 1377.48), (1330.52, 1389.22), (1330.52...",23746,0.459061,1943470,"[(529.6653366088867, 394.1354550962095), (522...."
1,1g_04.png,624,175683,"[(1354, 1377.48), (1330.52, 1389.22), (1330.52...",23746,0.862112,1943471,"[(564.6947244497446, 1107.9412689208984), (556..."
2,1g_04.png,624,175683,"[(1354, 1377.48), (1330.52, 1389.22), (1330.52...",23746,0.455114,1943472,"[(548.7361194776452, 1158.1390075683594), (550..."
3,1g_04.png,624,175683,"[(1354, 1377.48), (1330.52, 1389.22), (1330.52...",23746,0.256400,1943473,"[(524.8235282897949, 1337.475), (535.823528289..."
4,1g_04.png,624,175683,"[(1354, 1377.48), (1330.52, 1389.22), (1330.52...",23746,0.725020,1943474,"[(506.43187522888184, 1372.9502636137463), (50..."


In [15]:
# Compute Intersection union from the polygon coordinates 

def compute_iou(row):
    poly_manual = Polygon(row['xy_manual'])
    poly_pred = Polygon(row['xy_predicted'])
    if not poly_manual.is_valid or not poly_pred.is_valid:
        return 0.0
    intersection = poly_manual.intersection(poly_pred).area
    union = poly_manual.union(poly_pred).area
    return intersection / union if union > 0 else 0.0

merged_df['iou'] = merged_df.apply(compute_iou, axis=1)
merged_df.head()

ValueError: A linearring requires at least 4 coordinates.

In [16]:
from shapely.geometry import Polygon

def calculate_polygon_iou(polygon1_coords, polygon2_coords):
    """
    Calculates the Intersection over Union (IoU) of two polygons.

    Args:
        polygon1_coords (list of tuples): A list of (x, y) coordinates
                                          representing the vertices of the first polygon.
        polygon2_coords (list of tuples): A list of (x, y) coordinates
                                          representing the vertices of the second polygon.

    Returns:
        float: The IoU value between the two polygons. Returns 0.0 if the union area is zero.
    """
    # Create Shapely Polygon objects
    polygon1 = Polygon(polygon1_coords)
    polygon2 = Polygon(polygon2_coords)

    # Calculate the intersection area
    intersection_area = polygon1.intersection(polygon2).area

    # Calculate the union area
    union_area = polygon1.union(polygon2).area

    # Calculate IoU
    if union_area > 0:
        iou = intersection_area / union_area
    else:
        iou = 0.0  # Avoid division by zero if polygons don't overlap and have no area

    return iou

# Example Usage:
# Define polygon coordinates
poly1_coords = [(0, 0), (0, 2), (2, 2), (2, 0)]  # A square
poly2_coords = [(1, 1), (1, 3), (3, 3), (3, 1)]  # Another square, overlapping

iou_value = calculate_polygon_iou(poly1_coords, poly2_coords)
print(f"IoU between the two polygons: {iou_value}")

# Example with non-overlapping polygons
poly3_coords = [(0, 0), (0, 1), (1, 1), (1, 0)]
poly4_coords = [(2, 2), (2, 3), (3, 3), (3, 2)]

iou_non_overlapping = calculate_polygon_iou(poly3_coords, poly4_coords)
print(f"IoU between non-overlapping polygons: {iou_non_overlapping}")

IoU between the two polygons: 0.14285714285714285
IoU between non-overlapping polygons: 0.0


In [ ]:
merged_df['iou'] = [cal]

In [18]:
calculate_polygon_iou(
    merged_df["xy_manual"][0], merged_df["xy_predicted"][0]
)

0.0

In [25]:
merged_df["xy_manual"][1000]

[(845.272, 434.376),
 (833.532, 438.289),
 (833.532, 461.769),
 (837.445, 465.682),
 (849.185, 469.595),
 (857.012, 465.682),
 (860.925, 450.029),
 (849.185, 438.289),
 (849.185, 434.376)]

In [24]:
merged_df["xy_predicted"][0]

[(529.6653366088867, 394.1354550962095),
 (522.6653366088867, 398.7518572126116),
 (522.6653366088867, 413.1219870827415),
 (526.9708921644423, 421.39471435546875),
 (534.6653366088867, 430.01376197451634),
 (543.6653366088867, 430.42697241998485),
 (560.6653366088867, 420.9704719312263),
 (564.1480952295764, 416.39471435546875),
 (565.6653366088867, 400.1219870827415),
 (559.6653366088867, 394.19471435546876),
 (552.6653366088867, 392.39471435546875),
 (538.6653366088867, 392.54177317899814),
 (529.6653366088867, 394.1354550962095)]

In [ ]:
# Use the confidence value from the prediction as threshold
merged_df['tp'] = (merged_df['iou'] >= merged_df['confidence']).astype(int)
merged_df['fp'] = (merged_df['iou'] < merged_df['confidence']).astype(int)
# For FN, you would need to check for manual annotations with no predictions (not in merged_df)

In [ ]:
# Sum up TP, FP
TP = merged_df['tp'].sum()
FP = merged_df['fp'].sum()
# For FN, count manual annotations not matched (if needed)
FN = len(anno_df) - TP

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")